In [ ]:
import numpy as np
import pandas as pd

In [ ]:
rating_df = pd.read_csv('the-movies-dataset/ratings_small.csv')
rating_df.head()

In [ ]:
rating_df.shape

In [ ]:
rating_df.nunique()

In [ ]:
# sort data frame by timestemp for splitting
rating_df = rating_df.sort_values('timestamp')
rating_df.head()

In [ ]:
# map user id and movie id to integer starting from 0 to N (num of users) and M (num of movies)
from sklearn.preprocessing import LabelEncoder
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

user_ids = user_encoder.fit_transform(rating_df.userId)
movie_ids = movie_encoder.fit_transform(rating_df.movieId)

In [ ]:
num_train = int(len(user_ids) * 0.8)
train_user_ids = user_ids[:num_train]
train_movie_ids = movie_ids[:num_train]
train_ratings = rating_df.rating.values[:num_train]
val_user_ids = user_ids[num_train:]
val_movie_ids = movie_ids[num_train:]
val_ratings = rating_df.rating.values[num_train:]

In [ ]:
# set up user-movie matrix
num_users = user_ids.max() + 1
num_movies = movie_ids.max() + 1
user2movie = np.zeros([num_users, num_movies])
user2movie[train_user_ids, train_movie_ids] = train_ratings
user2movie

In [ ]:
# compute similarity
def pearson_correlation(x,y):
    '''
    Compute the pearson correlation for x and y.
    args:
        x: np.array
        y: np.array
    '''
    
    # set up filters for entries where x and y are both non zero
    filt = (x != 0) * (y != 0)
    
    # compute mean for x and y
    x_mean = x.sum() / x[x!=0].shape[0]
    y_mean = y.sum() / y[y!=0].shape[0]
    
    # consider only entries based on the previous filter
    x = x[filt]
    y = y[filt]
    
    # compute correlation
    corr = np.sum((x - x_mean) * (y-y_mean)) / (np.sum((y-y_mean) ** 2) * np.sum((x-x_mean) ** 2) ) ** 0.5
    
    return corr

def compute_user_similarity_matrix(user2movie):
    '''
    Compute user similarity matrix
    args:
        user2movie: np.array, user-movie rating matrix
    returns:
        similarity_matrix: np.array, user 2 user similarity
    
    '''
    # initialize similarity matrix
    similarity_matrix = np.zeros([num_users, num_users])
    
    for i in range(len(user2movie)):
        for j in range(i, len(user2movie)):
            
            # compute correlation
            corr = pearson_correlation(user2movie[i],user2movie[j])
            
            # store correlation in matrix
            similarity_matrix[i,j] = corr
            similarity_matrix[j,i] = corr
            
    return similarity_matrix

similarity_matrix = compute_user_similarity_matrix(user2movie)
similarity_matrix[:10]

In [ ]:
# set similarity with itself to 0
similarity_matrix[np.arange(num_users), np.arange(num_users)] = 0
similarity_matrix[np.isnan(similarity_matrix)] = 0
similarity_matrix[:10]

In [ ]:
def compute_ucf(user2movie, similarity_matrix):
    '''
    Compute prediction scores for all movies
    args:
        user2movie: np.array, user-movie rating matrix
    returns:
        scores: np.array, predicted scores of each video for the target user
    '''

    # compute mean rating yk, ignoring zero entries, shape:(num_users)
    mean_ratings = np.sum(user2movie, axis=1) / (user2movie != 0 ).sum(axis=1)
    
    # compute ykj - yk, shape:(num_users, num_movies)
    user2movie_diff = user2movie - np.expand_dims(mean_ratings, 1)
    
    # compute sum of similarities Σsimik, (num_users,)
    sim_sum = np.sum(np.abs(similarity_matrix), axis=1)
        
    # don't sum the unknown entries, set them to 0        
    user2movie_diff[np.where(user2movie == 0) ] = 0

    # compute weighted sum of rating diff (num_users, num_movies)
    weighted_sum = np.matmul(similarity_matrix, user2movie_diff) / np.expand_dims(sim_sum, 1)

    # add weighted sum to mean ratings
    scores =  weighted_sum + np.expand_dims(mean_ratings, 1)
    
    return  scores

predictions = compute_ucf(user2movie, similarity_matrix) 
predictions[:10]

In [ ]:
# credit to https://gist.github.com/bwhite/3726239
def dcg_at_k(r, k):
    '''
    Compute DCG
    args:
        r: np.array, to be evaluated
        k: int, number of entries to be considered
    
    returns:
        dcg: float, computed dcg
        
    '''
    r = r[:k]
    dcg = np.sum(r / np.log2(np.arange(2, len(r) + 2)))
    return dcg


In [ ]:
def ndcg_at_k(r, k, method=0):
    '''
    Compute NDCG
    args:
        r: np.array, to be evaluated
        k: int, number of entries to be considered
    
    returns:
        dcg: float, computed ndcg
        
    '''
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)

    return dcg_at_k(r, k) / dcg_max

In [ ]:
# compute average ndcg for all users
def evaluate_prediction(predictions):
    '''
    Return the average ndcg for each users
    args:
        predictions: np.array user-item predictions
    returns:
        ndcg: float, computed NDCG
    '''
    ndcgs = []
    # iterate
    for target_user in np.unique(val_user_ids):
        # get movie ids and ratings associated with the target user.
        target_val_movie_ids = val_movie_ids[val_user_ids == target_user] 
        target_val_ratings = val_ratings[val_user_ids == target_user] 
        
        # compute ndcg for this user
        ndcg = ndcg_at_k(target_val_ratings[np.argsort(-predictions[target_user][target_val_movie_ids])], k=30)
        ndcgs.append(ndcg)
    ndcg = np.mean(ndcgs)
    return ndcg
evaluate_prediction(predictions)

In [ ]:
def compute_item_similarity_matrix(user2movie):
    '''
    Compute item similarity matrix
    args:
        user2movie: np.array, user-movie rating matrix
    returns:
        similarity_matrix: np.array, item-item similarity
    
    '''
    # compute mean for each user
    x_mean = user2movie.sum(axis=0) / (user2movie!=0).sum(axis=0)
    
    # set up filter for zero entries
    filt = (user2movie==0)
    
    # compute rating difference
    rating_diff = user2movie - np.expand_dims(x_mean, axis=0)
    rating_diff[filt] = 0

    # compute similarity
    similarity_matrix = np.matmul(rating_diff.T, rating_diff) / (np.matmul(rating_diff.T  ** 2, (rating_diff != 0)) * np.matmul(rating_diff.T  ** 2, (rating_diff != 0)).T) ** 0.5

    return similarity_matrix

similarity_matrix = compute_item_similarity_matrix(user2movie)
similarity_matrix[:10]

In [ ]:
# set similarity with itself to 0
similarity_matrix[np.arange(num_movies), np.arange(num_movies)] = 0
similarity_matrix[np.isnan(similarity_matrix)] = 0
similarity_matrix[:10]

In [ ]:
def compute_icf(user2movie, similarity_matrix):
    '''
    Compute prediction scores for all movies with item-based CF.
    args:
        user2movie: np.array, user-movie rating matrix
    returns:
        scores: np.array, predicted scores of each video for the target user
    '''

    # compute mean rating yk, ignoring zero entries, shape:(num_movies)
    mean_ratings = np.sum(user2movie, axis=0) / (user2movie != 0 ).sum(axis=0)
    
    # compute ykj - yk, shape:(num_users, num_movies)
    user2movie_diff = user2movie - np.expand_dims(mean_ratings, axis=0)
    
    # compute sum of similarities Σsimik, (num_movies,)
    sim_sum = np.sum(np.abs(similarity_matrix), axis=1)
    
    # don't sum the unknown entries, set them to 0
    user2movie_diff[np.where(user2movie == 0) ] = 0
    
    # compute weighted sum of rating diff (num_users, num_movies)
    weighted_sum = np.matmul(user2movie_diff, similarity_matrix) / np.expand_dims(sim_sum, axis=0)

    # add weighted sum to mean ratings
    scores =  weighted_sum + np.expand_dims(mean_ratings, 0)
    
    return  scores

predictions = compute_icf(user2movie, similarity_matrix) 
predictions[:10]

In [ ]:
evaluate_prediction(predictions)